#### Score: 30.0505

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from scipy import stats

Using TensorFlow backend.


In [6]:
X_train = pd.read_csv('data_dengue/dengue_features_train.csv', index_col=None)
df_y_train = pd.read_csv('data_dengue/dengue_labels_train.csv', index_col=None)
X_test = pd.read_csv('data_dengue/dengue_features_test.csv', index_col=None)

In [7]:
X_train['week_start_date'] = pd.to_datetime(X_train['week_start_date'])
X_train['week_start_date'] = X_train['week_start_date'] - pd.to_datetime('1980-01-01')
X_train['week_start_date'] = pd.to_numeric(X_train['week_start_date'])/100000000000
X_train_new = X_train.fillna(X_train.mean())
X_train_sj=X_train_new[X_train_new.city=='sj']
X_train_iq=X_train_new[X_train_new.city=='iq']
X_train_sj = X_train_sj.drop(['city'], axis=1)
X_train_iq = X_train_iq.drop(['city'], axis=1)

In [8]:
X_test['week_start_date'] = pd.to_datetime(X_test['week_start_date'])
X_test['week_start_date'] = X_test['week_start_date'] - pd.to_datetime('1980-01-01')
X_test['week_start_date'] = pd.to_numeric(X_test['week_start_date'])/100000000000
X_test_new = X_test.fillna(X_test.mean())
X_test_sj=X_test_new[X_test_new.city=='sj']
X_test_iq=X_test_new[X_test_new.city=='iq']
X_test_sj = X_test_sj.drop(['city'], axis=1)
X_test_iq = X_test_iq.drop(['city'], axis=1)

In [9]:
X_train_sj_float = X_train_sj.astype('float64')
X_train_iq_float = X_train_iq.astype('float64')
X_test_sj_float = X_test_sj.astype('float64')
X_test_iq_float = X_test_iq.astype('float64')

In [10]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
X_train_sj_scaled = min_max_scaler.fit_transform(X_train_sj_float)
X_train_iq_scaled = min_max_scaler.fit_transform(X_train_iq_float)
X_test_sj_scaled = min_max_scaler.transform(X_test_sj_float)
X_test_iq_scaled = min_max_scaler.transform(X_test_iq_float)

In [11]:
print(X_train_sj_scaled[:5, :])
print(X_train_sj_scaled.shape)

[[0.         0.32692308 0.         0.58783972 0.62676332 0.52370092
  0.54188858 0.03179724 0.26100844 0.26918536 0.33993342 0.30769231
  0.45205479 0.05609115 0.3181382  0.03179724 0.29739227 0.35849109
  0.35968379 0.4403183  0.30337079 0.28205128 0.05230467]
 [0.         0.34615385 0.00106594 0.64041572 0.66981079 0.43529974
  0.49214226 0.05842294 0.36299338 0.38488784 0.5284738  0.47692308
  0.52054795 0.0314461  0.51021387 0.05842294 0.47346033 0.28598728
  0.53557312 0.34217507 0.56179775 0.56410256 0.02811376]
 [0.         0.36538462 0.00213187 0.48741177 0.70428426 0.42268026
  0.52666235 0.08842806 0.45402692 0.45690673 0.7103557  0.41538462
  0.64383562 0.04574934 0.73498766 0.08842806 0.66450897 0.26584733
  0.53557312 0.36339523 0.61797753 0.64102564 0.13533835]
 [0.         0.38461538 0.00319781 0.59454599 0.78500526 0.59484481
  0.67286631 0.03932412 0.48688113 0.51475797 0.69511127 0.55384615
  0.60273973 0.02436459 0.65265972 0.03932412 0.64176068 0.30209923
  0.640316

In [12]:
train_1s_sj = X_train_sj_scaled.reshape(X_train_sj_scaled.shape[0],1,X_train_sj_scaled.shape[1])
train_1s_iq = X_train_iq_scaled.reshape(X_train_iq_scaled.shape[0],1,X_train_iq_scaled.shape[1])

In [14]:
train_1s_sj.shape

(936, 1, 23)

In [15]:
data_y_1s_sj = np.array(df_y_train[df_y_train.city=='sj']["total_cases"])
data_y_1s_iq = np.array(df_y_train[df_y_train.city=='iq']["total_cases"])
data_y_1s_sj.shape

(936,)

In [16]:
model_1s_sj = Sequential()
model_1s_sj.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(1, 23)))
model_1s_sj.add(LSTM(50, activation='relu', return_sequences=True))
model_1s_sj.add(LSTM(30, activation='relu'))
model_1s_sj.add(Dense(1))
model_1s_sj.compile(optimizer='adam', loss='mae')

In [19]:
%%time
model_1s_sj.fit(train_1s_sj, data_y_1s_sj, epochs=400, verbose=0)

In [20]:
model_1s_iq = Sequential()
model_1s_iq.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(1, 23)))
model_1s_iq.add(LSTM(50, activation='relu', return_sequences=True))
model_1s_iq.add(LSTM(30, activation='relu'))
model_1s_iq.add(Dense(1))
model_1s_iq.compile(optimizer='adam', loss='mae')

In [21]:
%%time
model_1s_iq.fit(train_1s_iq, data_y_1s_iq, epochs=400, verbose=0)

Wall time: 44 s


In [22]:
data_1s_test_sj = X_test_sj_scaled.reshape(X_test_sj_scaled.shape[0],1,X_test_sj_scaled.shape[1])
data_1s_test_iq = X_test_iq_scaled.reshape(X_test_iq_scaled.shape[0],1,X_test_iq_scaled.shape[1])
data_1s_test_sj.shape

(260, 1, 23)

In [23]:
predict_sj = model_1s_sj.predict(data_1s_test_sj)

In [24]:
predict_iq = model_1s_iq.predict(data_1s_test_iq)

In [26]:
X_test_predict = pd.read_csv('data_dengue/dengue_features_test.csv', index_col=None)
result_columns_sj = X_test[X_test.city=='sj'][['city','year','weekofyear']]
result_cases_sj = pd.DataFrame(predict_sj)
result_cases_sj.columns = ['total_cases']
result_df_sj = pd.concat([result_columns_sj, result_cases_sj], axis=1)
result_df_sj['total_cases'] = result_df_sj['total_cases'].round().astype(int)
result_df_sj.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,4
1,sj,2008,19,4
2,sj,2008,20,4
3,sj,2008,21,6
4,sj,2008,22,6


In [31]:
result_columns_iq = X_test[X_test.city=='iq'][['city','year','weekofyear']]
result_cases_iq = pd.DataFrame(predict_iq)
result_cases_iq.columns = ['total_cases']
result_columns_iq = result_columns_iq.reset_index()
result_df_iq = pd.concat([result_columns_iq, result_cases_iq], axis=1)
result_df_iq['total_cases'] = result_df_iq['total_cases'].round().astype(int)
result_df_iq = result_df_iq.drop(['index'], axis=1)
result_df_iq.head()

,city,year,weekofyear,total_cases
0,iq,2010,26,2
1,iq,2010,27,2
2,iq,2010,28,2
3,iq,2010,29,2
4,iq,2010,30,2


In [32]:
result = pd.concat([result_df_sj,result_df_iq], axis=0)
result.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,4
1,sj,2008,19,4
2,sj,2008,20,4
3,sj,2008,21,6
4,sj,2008,22,6


In [33]:
result.to_csv('lstm_2_city_50_50_30.csv', index=False)